In [1]:
# !pip install transformers
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from datetime import datetime


time = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

In [2]:
texts, labels, files = [], [], ["combinedneg", "combinedneut2", "combinedpos"]
for i, file in enumerate(files):
    with open(f"E:/Emodim/data/tr/{file}.txt", "r") as f:
        lines = f.read().splitlines()
        texts.extend(lines), labels.extend([i]*len(lines))
print(texts[:5], labels[:5])

['ei mitään muttia.', 'älä anna hänen määräillä sinua!', 'laske aseet maahan.', 'maksut pitää hoitaa ajallaan.', 'lntiaanirenki, juoppo pyssymies, seksihullu ja eno!'] [0, 0, 0, 0, 0]


In [3]:
# !pip install sklearn
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.1)


train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels))

In [6]:
from transformers import TFBertForSequenceClassification


model = TFBertForSequenceClassification.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

"""
model.compile(
              optimizer = tf.keras.optimizers.experimental.AdamW(learning_rate=5e-5, weight_decay=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = tf.keras.metrics.SparseCategoricalAccuracy())
history = model.fit(train_dataset, batch_size=16, callbacks=[early_stop], epochs=10)
"""
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)) # can also use any keras loss fn
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


AttributeError: in user code:

    File "E:\Miniconda\envs\tf\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "E:\Miniconda\envs\tf\lib\site-packages\transformers\modeling_tf_utils.py", line 947, in compute_loss  *
        return super().compute_loss(*args, **kwargs)
    File "E:\Miniconda\envs\tf\lib\site-packages\keras\engine\training.py", line 948, in compute_loss  **
        return self.compiled_loss(
    File "E:\Miniconda\envs\tf\lib\site-packages\keras\engine\compile_utils.py", line 199, in __call__
        y_t, y_p, sw = match_dtype_and_rank(y_t, y_p, sw)
    File "E:\Miniconda\envs\tf\lib\site-packages\keras\engine\compile_utils.py", line 726, in match_dtype_and_rank
        if ((y_t.dtype.is_floating and y_p.dtype.is_floating) or

    AttributeError: 'NoneType' object has no attribute 'dtype'


In [ ]:
modelpath = f'E:\Emodim\data\others\\fine_tuned_finBERT_tf_{time}'
model.save(modelpath)

In [ ]:
import_model = tf.keras.models.load_model(modelpath)
import_model.summary()

In [ ]:
import random
import numpy as np
n = 50
inputs = []
with open(f"D:\\Work\\Data\\s24_2001_sentences_shuffled_slice.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    random.shuffle(lines)
    for line in lines:
        inputs.append(line.strip('\n'))

In [ ]:
preds = []
for inp in inputs[:n]:
    input_text_tokenized = tokenizer.encode(inp, truncation=True, padding=True, return_tensors="tf")
    prediction = model(input_text_tokenized)
    prediction_logits = prediction[0]
    prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()
    print(f'Sentence: {inp}, Probabilities: {prediction_probs}')